<a href="https://colab.research.google.com/github/imankassim/helmet_detection/blob/master/Object_Detection_Training_Helmet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Download the project

In [38]:
import torch

In [1]:
!wget -c 'https://helmet-detection-computer-vision.s3.eu-west-2.amazonaws.com/data.zip'


--2020-09-07 16:30:27--  https://helmet-detection-computer-vision.s3.eu-west-2.amazonaws.com/data.zip
Resolving helmet-detection-computer-vision.s3.eu-west-2.amazonaws.com (helmet-detection-computer-vision.s3.eu-west-2.amazonaws.com)... 52.95.150.38
Connecting to helmet-detection-computer-vision.s3.eu-west-2.amazonaws.com (helmet-detection-computer-vision.s3.eu-west-2.amazonaws.com)|52.95.150.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 496206 (485K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 484.58K  --.-KB/s    in 0.04s   

2020-09-07 16:30:28 (11.3 MB/s) - ‘data.zip’ saved [496206/496206]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/coco/
   creating: data/coco/annotations/
  inflating: data/coco/annotations/instances_test2017.json  
  inflating: data/coco/annotations/instances_train2017.json  
  inflating: data/coco/annotations/instances_val2017.json  
  inflating: data/coco/labelme2coco.py  
   creating: data/coco/test2017/
  inflating: data/coco/test2017/25.jpg  
  inflating: data/coco/test2017/25.json  
  inflating: data/coco/test2017/26.jpg  
  inflating: data/coco/test2017/26.json  
  inflating: data/coco/test2017/27.jpg  
  inflating: data/coco/test2017/27.json  
  inflating: data/coco/test2017/28.jpg  
  inflating: data/coco/test2017/28.json  
  inflating: data/coco/test2017/29.jpg  
  inflating: data/coco/test2017/29.json  
   creating: data/coco/train2017/
  inflating: data/coco/train2017/0.jpg  
  inflating: data/coco/train2017/0.json  
  inflating: data/coco/train2017/1.jpg  
  inflating: data/coco/train2017/1.json  
  inflating: data/coco/train2017/10.jpg  
  infla

### Change directory into the project

In [3]:
pwd

'/content'

### Clone the MMDetection project

In [4]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13137 (delta 1), reused 1 (delta 0), pack-reused 13128
Receiving objects: 100% (13137/13137), 13.71 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (8873/8873), done.


### Move the data into the mmdetection directory

In [5]:
mv data/ mmdetection/

In [6]:
pwd

'/content'

### Change into the mmdetection directory

In [7]:
cd mmdetection/

/content/mmdetection


### Build and setup MMDetection

In [8]:
!sudo pip install -r requirements/build.txt
!sudo pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"
!sudo pip install -v -e .
!sudo pip install mmcv-full

  Cloning https://github.com/open-mmlab/cocoapi.git to /tmp/pip-req-build-ljlmew2u
  Running command git clone -q https://github.com/open-mmlab/cocoapi.git /tmp/pip-req-build-ljlmew2u
  Created wheel for pycocotools: filename=pycocotools-12.0-cp36-cp36m-linux_x86_64.whl size=266753 sha256=ec2fbece02cb6b23780852bb8fc2c2160a1c2e85c1350fa6d129528799fc6cd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-wwk6ofhk/wheels/cd/f6/de/018ccc2d175046c612e93b42a169cd1ab7563d61581cfba8df
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.1
    Uninstalling pycocotools-2.0.1:
      Successfully uninstalled pycocotools-2.0.1
Created temporary directory: /tmp/pip-ephem-wheel-cache-jyiybrr5
Created temporary directory: /tmp/pip-req-tracker-vabvx3lr
Created requirements tracker '/tmp/pip-req-tracker-vabvx3lr'
Created temporary directory: /tmp/pip-install-mt5sqpxy
Obtaining file:///content/mmdetection
  Added file:///content/mmdetection to build tracker '/tmp/pip-req-tracker-v

In [24]:
!sudo pip install mmcv==1.1.1

     |████████████████████████████████| 245kB 6.5MB/s 
  Created wheel for mmcv: filename=mmcv-1.1.1-cp36-cp36m-linux_x86_64.whl size=393324 sha256=578e6f590bf2574e730d5cdefa499e27890073069815579593c6af1d246ed3dd
  Stored in directory: /root/.cache/pip/wheels/af/b2/4b/ab2422df0e82dcaa43835609e2fc01b5f454e8f5346a0cb26b
Successfully built mmcv
  Found existing installation: mmcv 1.0.5
    Uninstalling mmcv-1.0.5:
      Successfully uninstalled mmcv-1.0.5


## Step 2: Setup code structure

### Setup the libraries we need

In [10]:
import os
from os.path import exists, join, basename, splitext
import json
import pandas as pd
import numpy as np

### Setup model structure for choosing the file to run

In [11]:
# This provides you a link to the file you want to use and the selected model you want to build

MODELS_CONFIG = {
    'faster_rcnn_r50_fpn_1x_coco':{
        'config_file': 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
    }
}

### Setup the number of epochs and model structure

In [12]:
# Pick the model you want to use

# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_r50_fpn_1x_coco' 

# Total training epochs if you want to update the number of training epoch
total_epochs = 12

# Recommended resize value
image_size = (1333, 800)

# Name of the config file.
config_file = MODELS_CONFIG[selected_model]['config_file']

### Edit files to update classes to match our own class names


> MMDetection trains using 80 classes, our project only has one class, so we 
update some files to update class names and number of classes

In [13]:
# We want to get the classes we are annotating from
annotation_path = os.path.join("", "data/coco/annotations", "instances_train2017.json")
json_file = open(annotation_path)
coco = json.load(json_file)
print(coco["categories"])
classes_names = [category["name"] for category in coco["categories"]]
print(classes_names)

[{'supercategory': 'helmet', 'id': 0, 'name': 'helmet'}]
['helmet']


### Update the "/mmdet/datasets/coco.py" file to update the class names

In [14]:
coco_dataset_class_file = "mmdet/datasets/coco.py"
import re
fname = coco_dataset_class_file
with open(fname) as f:
  s = f.read()
  s = re.sub('CLASSES = \(.*?\)',
               'CLASSES = ({})'.format("".join(["\'{}\',".format(name) for name in classes_names])), s, flags=re.S)
with open(fname, 'w') as f:
    f.write(s)

### Setup the base code from where all its code stems from


In [15]:
base_config_fname = 'configs/_base_/models/faster_rcnn_r50_fpn.py'

### Update the number of classes we are working on in the base file


In [16]:
fname = base_config_fname
with open(fname) as f:
    s = f.read()
    # Update `num_classes` including `background` class.
    s = re.sub('num_classes=.*?,',
               'num_classes={},'.format(len(classes_names)), s)
with open(fname, 'w') as f:
    f.write(s)

### Create helper function

In [17]:
def replace_second_occurrence(text, find_this, replace_with):
    return text.replace(find_this, replace_with, 2).replace(replace_with, find_this, 1)

### Update the image size, we also add the update the test json files


In [18]:
coco_instance_segmentation_fname = 'configs/_base_/datasets/coco_instance.py'
fname = coco_instance_segmentation_fname
with open(fname) as f:
    s = f.read()
    s = re.sub('img_scale=\(.*?,.*?\)',
               'img_scale={}'.format(image_size), s)
    s = replace_second_occurrence(s, 'annotations/instances_val2017.json', 'annotations/instances_test2017.json')
    s = replace_second_occurrence(s, 'val2017/', 'test2017/')
with open(fname, 'w') as f:
    f.write(s)
    
coco_detection_fname = 'configs/_base_/datasets/coco_detection.py'
fname = coco_detection_fname
with open(fname) as f:
    s = f.read()
    s = re.sub('img_scale=\(.*?,.*?\)',
               'img_scale={}'.format(image_size), s)
    s = replace_second_occurrence(s, 'annotations/instances_val2017.json', 'annotations/instances_test2017.json')
    s = replace_second_occurrence(s, 'val2017/', 'test2017/')
with open(fname, 'w') as f:
    f.write(s)

### Update the number of epochs and learning rate

In [19]:
base_schedule_fname = 'configs/_base_/schedules/schedule_1x.py'
lr = 0.02

fname = base_schedule_fname
with open(fname) as f:
    s = f.read()
    s = re.sub('total_epochs = \d+',
               'total_epochs = {}'.format(total_epochs), s)
    s = re.sub('lr=\d+.\d+',
               'lr={}'.format(lr), s)
with open(fname, 'w') as f:
    f.write(s)

### Add a pretrained model

In [25]:
# If you want to use a COCO pretrained model, you need a link to the model. 
# You replace the None in load_from with the link to the pretrained model. 
# We do not have a pretrained model for ResNet18 on COCO, 
# so do not update the load_from, leave it as None

%%writefile configs/_base_/default_runtime.py
checkpoint_config = dict(interval=1)
# yapf:disable
log_config = dict(
    interval=50,
    hooks=[
        # dict(type='TextLoggerHook'),
        # dict(type='TensorboardLoggerHook')
    ])
# yapf:enable
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
resume_from = None
workflow = [('train', 1)]

Overwriting configs/_base_/default_runtime.py


## Step 3: Training Time!!!

### Time to train the model

In [26]:
!python tools/train.py {config_file}

2020-09-07 16:45:08,442 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.6.9 (default, Jul 17 2020, 12:50:27) [GCC 8.4.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.1, V10.1.243
GPU 0: Tesla K80
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.6.0+cu101
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2019.0.5 Product Build 20190808 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.5.0 (Git Hash e2ac1fac44c5078ca927cb9b90e1b3066a0b2ed0)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75
  - CuDNN 7.6.3


### Confirmed the trained model exists

In [27]:
# Confirm the latest trained model exists
work_dir = "work_dirs/" + selected_model
checkpoint_file = os.path.join(work_dir, "latest.pth")
assert os.path.isfile(
    checkpoint_file), '`{}` not exist'.format(checkpoint_file)
checkpoint_file = os.path.abspath(checkpoint_file)
checkpoint_file

'/content/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_coco/latest.pth'

### Evaluate the results of the model

In [28]:
# Evaluate your model by bbox for bounding box
!python tools/test.py {config_file} {checkpoint_file} --eval bbox

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[                                                  ] 0/5, elapsed: 0s, ETA:/content/mmdetection/mmdet/core/post_processing/bbox_nms.py:52: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  labels = valid_mask.nonzero()[:, 1]
[>>] 5/5, 2.7 task/s, elapsed: 2s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.672
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 1.000
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.

### Step 4: Time to test the model

### Setup library and functions for testing the model

In [29]:
import glob
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Collect the list of images to test. You can update this list with the files you want to test. It can be a single image or multiple images
files_for_test = glob.glob("data/coco/test2017/*.jpg")

### Create a helper function to test our model

In [30]:
# We create a helper method that takes an image and returns its prediction. We save a copy of the image appending result at the beginning of the original image name
from IPython.display import Image
def show_result(img_path, score_thr):
  
  # build the model from a config file and a checkpoint file
  model = init_detector(config_file, checkpoint_file, device='cuda:0')

  img_result_path = 'result_' + img_path.split("/")[-1]

  print("Result: ", img_result_path)

  result = inference_detector(model, img_path)
  res2 = model.show_result(img_path, result,
            score_thr=score_thr, thickness = 4, 
            bbox_color = "green",  
            font_scale=1.0, 
            out_file=img_result_path, 
            text_color="green"
            )
  
  # Show the image with bbox overlays.
  Image(filename=img_result_path) 

In [ ]:
pwd

In [31]:
# Testing the above method for only one image
score_thr = 0.5 # Choose the threshold of the accuracy you want
img = 'data/coco/test2017/27.jpg' # Location of the image
show_result(img, score_thr) # Call the method passing in the necessary parameters

Result:  result_27.jpg


/content/mmdetection/mmdet/core/post_processing/bbox_nms.py:52: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  labels = valid_mask.nonzero()[:, 1]


### Test the prediction on the test models

In [32]:
# To test on images in a list
score_thr = 0.5 # Choose accuracy threshold
for file in files_for_test:
  show_result(file, score_thr)

Result:  result_29.jpg
Result:  result_28.jpg
Result:  result_27.jpg
Result:  result_25.jpg
Result:  result_26.jpg


In [33]:
cd mmdetection/

[Errno 2] No such file or directory: 'mmdetection/'
/content/mmdetection


In [41]:
img = 'data/coco/val2017/20.jpg' # ENTER IMAGE HERE
show_result(img, score_thr) # Call the method passing in the necessary parameters

Result:  result_20.jpg
